#Day 09 of 14 Days Databricks Challenge

In [0]:
CREATE DATABASE IF NOT EXISTS gold;


In [0]:
SHOW DATABASES;


In [0]:
DROP TABLE IF EXISTS default.products;
DROP TABLE IF EXISTS gold.products;


In [0]:
SHOW TABLES IN default;

In [0]:
SHOW TABLES IN gold;


In [0]:
CREATE TABLE default.gold_product_metrics AS
SELECT
  product_id,
  category_code,
  CAST(event_time AS DATE) AS event_date,

  COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
  COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,

  SUM(
    CASE
      WHEN event_type = 'purchase' THEN price
      ELSE 0
    END
  ) AS revenue

FROM default.silver_events
GROUP BY
  product_id,
  category_code,
  CAST(event_time AS DATE);


In [0]:
DESCRIBE default.gold_product_metrics;


In [0]:
SELECT COUNT(*) FROM default.gold_product_metrics;


In [0]:
SELECT *
FROM default.gold_product_metrics
ORDER BY revenue DESC
LIMIT 10;


In [0]:
-- LOG: Check date coverage
SELECT
  MIN(event_date) AS min_date,
  MAX(event_date) AS max_date,
  COUNT(DISTINCT event_date) AS total_days
FROM default.gold_product_metrics;


In [0]:
-- Daily revenue trend
SELECT
  event_date,
  SUM(revenue) AS daily_revenue
FROM default.gold_product_metrics
GROUP BY event_date
ORDER BY event_date;


In [0]:
-- LOG: Validate daily aggregation
WITH daily AS (
  SELECT
    event_date,
    SUM(revenue) AS rev
  FROM default.gold_product_metrics
  GROUP BY event_date
)
SELECT COUNT(*) AS days_available
FROM daily;


In [0]:
-- Revenue + 7-day moving average
WITH daily AS (
  SELECT
    event_date,
    SUM(revenue) AS rev
  FROM default.gold_product_metrics
  GROUP BY event_date
)
SELECT
  event_date,
  rev,
  AVG(rev) OVER (
    ORDER BY event_date
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS ma7
FROM daily
ORDER BY event_date;


In [0]:
-- LOG: Number of products
SELECT COUNT(DISTINCT product_id) AS total_products
FROM default.gold_product_metrics;


In [0]:
-- Top products
SELECT
  product_id,
  SUM(revenue) AS total_revenue
FROM default.gold_product_metrics
GROUP BY product_id
ORDER BY total_revenue DESC
LIMIT 20;


In [0]:
-- LOG: Funnel totals
SELECT
  SUM(views) AS total_views,
  SUM(purchases) AS total_purchases
FROM default.gold_product_metrics;


In [0]:
-- Conversion funnel
SELECT
  SUM(views) AS views,
  SUM(purchases) AS purchases,
  ROUND(
    SUM(purchases) * 100.0 / NULLIF(SUM(views), 0),
    2
  ) AS conversion_rate
FROM default.gold_product_metrics;


In [0]:
-- LOG: Available categories
SELECT DISTINCT category_code
FROM default.gold_product_metrics
ORDER BY category_code;


In [0]:
-- Category performance
SELECT
  category_code,
  SUM(views) AS views,
  SUM(purchases) AS purchases,
  ROUND(
    SUM(purchases) * 100.0 / NULLIF(SUM(views), 0),
    2
  ) AS conversion_rate,
  SUM(revenue) AS revenue
FROM default.gold_product_metrics
GROUP BY category_code
ORDER BY revenue DESC;


In [0]:
-- LOG: Filter domain check
SELECT
  COUNT(DISTINCT event_date) AS date_values,
  COUNT(DISTINCT category_code) AS category_values
FROM default.gold_product_metrics;


In [0]:
-- LOG: Final health check
SELECT
  COUNT(*) AS rows,
  SUM(revenue) AS total_revenue,
  SUM(purchases) AS total_purchases
FROM default.gold_product_metrics;


In [0]:
-- LOG: Check daily revenue aggregation
SELECT COUNT(*) AS days_available
FROM (
  SELECT event_date
  FROM gold.products
  GROUP BY event_date
);
